# Stochastic Optimization for Air Travel

Optimization tries many different solutions and scores the best one to use. One simple examples is using random guesses and scoring to determine which one is best. There are a number of issues that make this approach ineffective - one of which processing time. There are many problems where there just are too many options to consider. 

This example plans a trip for a group of people (all of whom live in different cities). They want to meet in New York. However, they want to arrive and leave New York on the same day. This example will try to find the optimal flights to take to achieve this goal and minimize costs. 

Below will import all the packages we need including time, random, and math. We will also define our "group" of people and their respective cities. They all want to arrive at Lagaurdia Airport in NYC. 

In [11]:
import time
import random
import math

people = [('Seymour','BOS'),
          ('Franny','DAL'),
          ('Zooey','CAK'),
          ('Walt','MIA'),
          ('Buddy','ORD'),
          ('Les','OMA')]
# Laguardia
destination='LGA'

Next, we are going to convert our schedule.txt file into a dictionary data object that we can use. 

In [13]:
flights={}

for line in file('schedule.txt'):
    #import a) origin, b) destination, c) arrive, and d) price. In the .txt file, 
    #they are split by a ,.
  origin,dest,depart,arrive,price=line.strip().split(',')
    #.setdefault origin and destination an empty array
  flights.setdefault((origin,dest),[])

  # Add details to the list of possible flights
  flights[(origin,dest)].append((depart,arrive,int(price)))

The above is a standard way to convert/clean a .txt file into a usable data object. An additional step of cleaning will also convert the arrival and depart times to minutes as shown below. 

In [4]:
def getminutes(t):
  x=time.strptime(t,'%H:%M')
  return x[3]*60+x[4]

In [21]:
def printschedule(r):
    #we want the range of r/2 since there is an arrival flight and return flight. 
  for d in range(len(r)/2):
    #take the first column in our people array. 
    name=people[d][0]
    #take the second column in our people array.
    origin=people[d][1]
    out=flights[(origin,destination)][int(r[d*2])]
    ret=flights[(destination,origin)][int(r[d*2+1])]
    print '%10s%10s %5s-%5s $%3s %5s-%5s $%3s' % (name,origin,
                                                  out[0],out[1],out[2],
                                                  ret[0],ret[1],ret[2])

In [35]:
s=[1,4,3,2]

In [36]:
printschedule(s)

   Seymour       BOS  8:04-10:11 $ 95 12:08-14:05 $142
    Franny       DAL 10:30-14:57 $290  9:49-13:51 $229


It is important to note that the above does NOT show the optimal arrival/departures. We haven't reached that point yet. It just shows a random schedule we inputed. We could have very well at s = [1,2,3,4]. See below for how this would look:

In [48]:
r = [1,2,3,4,5,6,7,8]

In [49]:
printschedule(r)

   Seymour       BOS  8:04-10:11 $ 95  9:58-11:18 $130
    Franny       DAL 10:30-14:57 $290 12:20-16:34 $500
     Zooey       CAK 13:40-15:38 $137 15:50-18:45 $243
      Walt       MIA 17:07-20:04 $291 18:07-21:30 $355


Now lets set up our cost function. There is no good or bad way to set up a cost. Some of the attributes we can look at are price, travel time, waiting time, departure time, and car rental period. Usually we optimize by have the lowest cost at the end of the day. 

In [32]:
outbound=flights[(origin,destination)]

In [33]:
outbound

[('6:08', '8:06', 224),
 ('8:27', '10:45', 139),
 ('9:15', '12:14', 247),
 ('10:53', '13:36', 189),
 ('12:08', '14:59', 149),
 ('13:40', '15:38', 137),
 ('15:23', '17:25', 232),
 ('17:08', '19:08', 262),
 ('18:35', '20:28', 204),
 ('20:30', '23:11', 114)]

In [37]:
def schedulecost(sol):
  totalprice=0
  latestarrival=0
  earliestdep=24*60

  for d in range(len(sol)/2):
    # Get the inbound and outbound flights
    origin=people[d][1]
    outbound=flights[(origin,destination)][int(sol[d*2])]
    returnf=flights[(destination,origin)][int(sol[d*2+1])]
    
    # Total price is the price of all outbound and return flights
    totalprice+=outbound[2]
    totalprice+=returnf[2]
    
    # Track the latest arrival and earliest departure
    if latestarrival<getminutes(outbound[1]): latestarrival=getminutes(outbound[1])
    if earliestdep>getminutes(returnf[0]): earliestdep=getminutes(returnf[0])
  
  # Every person must wait at the airport until the latest person arrives.
  # They also must arrive at the same time and wait for their flights.
  totalwait=0  
  for d in range(len(sol)/2):
    origin=people[d][1]
    outbound=flights[(origin,destination)][int(sol[d*2])]
    returnf=flights[(destination,origin)][int(sol[d*2+1])]
    totalwait+=latestarrival-getminutes(outbound[1])
    totalwait+=getminutes(returnf[0])-earliestdep  
  
  return totalwait+totalprice

There is a lot going on in the cost function above. Let's break it down. The argument that this schedulecost() recieves is "Sol" or our vector that was fed into printschedule(s) previously defined. 

The first components in our loop are fairly straightforward. Note: range() is used in for-loops to state how many times we want to loop. It's not enough to say "for 'd' in (len(sol/2)) because (len(sol/2)) is an integer and we don't use an integer to establish a for-loop. We use an array of numbers (i.e. [0,1,2,3]). 

"d" then is defined based on this range. So let's say we used s=[1,4,3,2]. (len(sol/2)) makes this 2. range((len(sol/2))) makes this [0,1] which makes 'd' 0 and 1. So, when we get to our 2nd iteration we have origin = people[d][1] that means people[1][1]. Then, that means go to our people array go to the second (2nd) row and second (2nd) column. Use that value. It's 'DAL' --> which is the Dallas airport.

Let's follow this thread along. 'd' is still 1. We get to the next line, and flights [(origin,destination)][int(s[1*2])] means pull the origin and destination times and the price for the flights array for the S-row. Here, the S-row will NOT be 2. It'll be the third column whatever our s-vector was. In this case the third column value is 3. So, now we can pull the 3rd row of flights data. 

If you remember, we defined s=[1,4,3,2] as s = [departing, arriving, departing, arriving] and so and so forth. So the 1st value is outbound for Seymour and our third value is outbound for Fanny. 

Return flight information is defined the same way, but as you would expect we just add one to get the arriving: sol[d*2+1]. This record is defined as our return value. 

Next, we have a conditional. "latestarrival" was defined as 0 earlier in the function. So, this line converts all the outbound times to integer minutes with our getminutes(). The same applies to earliest departure. 

Finally, we are already to incorporate some type of cost. This is done by calculating the difference between the latest arrival and the latest arrival under the proposed schedule. The bigger the proposed schedule is off, the bigger the cost will be.

The same holds true for departures. We then sum of the total waits to get our cost. 

In [38]:
schedulecost(s)

1181

Now, we are able to evaluate our costs for any given schedule. We tried s=[1,4,3,2] above. Which means Seymour took the 2nd flight from Boston and the fifth flight back home. Then Franny took the 4th flight from DAL and 3rd flight back home.

When Seymour and Franny made this choice, it cost them $1,181. 

Let's have Seymour and Franny take first flight all the time. So s_trial2 = [0,0,0,0]. This seems like it would lower costs right? Since the first flight out tends to be super early in the morning and nobody likes to wake early? idk.

In [39]:
s_trial2 = [0,0,0,0]

In [40]:
schedulecost(s_trial2)

965

I guessed right. Seymour and Franny saved $216 by always taking the first flight out. What if they took the 10th flight all the time? or s_trial3 = [9,9,9,9].

In [42]:
s_trial3 = [9,9,9,9]

In [43]:
schedulecost(s_trial3)

1295

This cost us $114 more...So in our three trial batches, it looks like we see earlier flights to be cheaper. However, the problem is that we manually had to sample these schedules. What if the 2nd flight out and the 14th flight back turn out to be the best? What if there were a way to try out all the other combinations? There is. We'll cover it in the next section using a random search method. It's simple; it uses brute force; it's the least effective optimization, but it serves as a good baseline to compare against other optimization algos we may throw at the prob. 

In [7]:
def randomoptimize(domain,costf):
  best=999999999
  bestr=None
  for i in range(0,1000):
    # Create a random solution
    r=[float(random.randint(domain[i][0],domain[i][1])) 
       for i in range(len(domain))]
    
    # Get the cost
    cost=costf(r)
    
    # Compare it to the best one so far
    if cost<best:
      best=cost
      bestr=r 
  return r

In [8]:
def hillclimb(domain,costf):
  # Create a random solution
  sol=[random.randint(domain[i][0],domain[i][1])
      for i in range(len(domain))]
  # Main loop
  while 1:
    # Create list of neighboring solutions
    neighbors=[]
    
    for j in range(len(domain)):
      # One away in each direction
      if sol[j]>domain[j][0]:
        neighbors.append(sol[0:j]+[sol[j]+1]+sol[j+1:])
      if sol[j]<domain[j][1]:
        neighbors.append(sol[0:j]+[sol[j]-1]+sol[j+1:])

    # See what the best solution amongst the neighbors is
    current=costf(sol)
    best=current
    for j in range(len(neighbors)):
      cost=costf(neighbors[j])
      if cost<best:
        best=cost
        sol=neighbors[j]

    # If there's no improvement, then we've reached the top
    if best==current:
      break
  return sol

In [9]:
def annealingoptimize(domain,costf,T=10000.0,cool=0.95,step=1):
  # Initialize the values randomly
  vec=[float(random.randint(domain[i][0],domain[i][1])) 
       for i in range(len(domain))]
  
  while T>0.1:
    # Choose one of the indices
    i=random.randint(0,len(domain)-1)

    # Choose a direction to change it
    dir=random.randint(-step,step)

    # Create a new list with one of the values changed
    vecb=vec[:]
    vecb[i]+=dir
    if vecb[i]<domain[i][0]: vecb[i]=domain[i][0]
    elif vecb[i]>domain[i][1]: vecb[i]=domain[i][1]

    # Calculate the current cost and the new cost
    ea=costf(vec)
    eb=costf(vecb)
    p=pow(math.e,(-eb-ea)/T)

    # Is it better, or does it make the probability
    # cutoff?
    if (eb<ea or random.random()<p):
      vec=vecb      

    # Decrease the temperature
    T=T*cool
  return vec

In [10]:
def geneticoptimize(domain,costf,popsize=50,step=1,
                    mutprob=0.2,elite=0.2,maxiter=100):
  # Mutation Operation
  def mutate(vec):
    i=random.randint(0,len(domain)-1)
    if random.random()<0.5 and vec[i]>domain[i][0]:
      return vec[0:i]+[vec[i]-step]+vec[i+1:] 
    elif vec[i]<domain[i][1]:
      return vec[0:i]+[vec[i]+step]+vec[i+1:]
  
  # Crossover Operation
  def crossover(r1,r2):
    i=random.randint(1,len(domain)-2)
    return r1[0:i]+r2[i:]

  # Build the initial population
  pop=[]
  for i in range(popsize):
    vec=[random.randint(domain[i][0],domain[i][1]) 
         for i in range(len(domain))]
    pop.append(vec)
  
  # How many winners from each generation?
  topelite=int(elite*popsize)
  
  # Main loop 
  for i in range(maxiter):
    scores=[(costf(v),v) for v in pop]
    scores.sort()
    ranked=[v for (s,v) in scores]
    
    # Start with the pure winners
    pop=ranked[0:topelite]
    
    # Add mutated and bred forms of the winners
    while len(pop)<popsize:
      if random.random()<mutprob:

        # Mutation
        c=random.randint(0,topelite)
        pop.append(mutate(ranked[c]))
      else:
      
        # Crossover
        c1=random.randint(0,topelite)
        c2=random.randint(0,topelite)
        pop.append(crossover(ranked[c1],ranked[c2]))
    
    # Print current best score
    print scores[0][0]
    
  return scores[0][1]